<a href="https://colab.research.google.com/github/ahlqui/VeloxChemColabs/blob/main/MolecularDynamicsConformerSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>




# Generate a force field from XYZ and run dynamics


This notebook requires that you have a molecular structure. Two options:

Define Molecule as XYZ-coordinates: Can be generatred with free software such as www.avogadro.cc

Define Molecule with SMILES code, A molecule can be defined using a SMILES code (example below). We have two suggested ways to generate smiles from structure. 1) Sketch your molecule at https://www.rcsb.org/chemical-sketch and the SMILES code will be shown right below the structure. 2) Build your molecule at https://molview.org/ , go to Tools/Information Card and it will show you the SMILES code. Just copy/paste it into the SMILES box below.

Example of xyz-coordinates for CO2:

O 0.00 0.00 0.00

C 0.00 0.00 1.20

O 0.00 0.00 2.40

Example of SMILES for CO2:

O=C=O

In [1]:
#@title Install VeloxChem and dependencies (ca 2 min)
#@markdown Before you can run anything a few things need to be installed. It all runs on the Google Colab server an you do not need to install anything on your computer.
%%capture
import sys

! wget -qnc https://github.com/conda-forge/miniforge/releases/latest/download/Miniforge3-Linux-x86_64.sh
! bash Miniforge3-Linux-x86_64.sh -bfp /usr/local
! mamba config --set auto_update_conda false
import sys
sys.path.append('/usr/local/lib/python3.11/site-packages/')
!mamba install -y -c conda-forge -c veloxchem veloxchem xtb-python dftd4-python py3Dmol MDAnalysis openmm matplotlib python=3.11
! pip install rdkit
import veloxchem as vlx

In [2]:
xyz_coordinates = """
N     0.4785967928750  -0.4330741880750  -0.4934317502006
H     1.3794564278886   0.0429751463168  -0.5120458348956
C     0.4573285404340  -1.3154740355836  -1.6452845209868
H     1.2794157350200  -2.0270971721773  -1.5868316520123
H    -0.4697908768414  -1.8882598290096  -1.6558792536570
C     0.3747047426388  -1.2102491878280   0.7331072010194
H    -0.4082292852521  -1.9567463178277   0.6148319854093
H     1.2952953868059  -1.7540437197433   0.9183116804327
C     0.6956425554192   0.9059967227241  -5.3588904651214
C     1.6690431285267  -0.0704102447688  -5.0614336089256
C     1.5905279342425  -0.7922292640998  -3.8527793116028
C     0.5421211238334  -0.5430166236398  -2.9451176135833
C    -0.4312572408155   0.4285446612223  -3.2497814984180
C    -0.3564249048118   1.1570067527526  -4.4544417171718
H     0.7549521548685   1.4652126122086  -6.2812972715425
H     2.3338053657174  -1.5390048087124  -3.6163728176857
H    -1.2215586002930   0.6165771400284  -2.5371827271696
C    -0.5809783149118   1.1061887532548   4.2819306994214
C    -1.0539691464599  -0.2152180430193   4.1484967454886
C    -0.7402866542775  -0.9585904738513   2.9917812697427
C     0.0501603450518  -0.3918335577218   1.9715484114033
C     0.5329231859523   0.9234697295847   2.1220017776407
C     0.2096151223583   1.6793509886579   3.2655299818144
H    -0.8298576406810   1.6816374807225   5.1614537759970
H    -1.1155297675244  -1.9639771384900   2.8737258346041
H     1.1311551788061   1.3586475153088   1.3389498331805
C     2.7985412681099  -0.3440921389086  -6.0320361103160
H     3.0137177612643  -1.4119025048439  -6.0784904425001
H     2.5433990952730  -0.0090827377477  -7.0376742707821
H     3.7013243404231   0.1769923076871  -5.7132670703364
C    -1.3956677617432   2.2110713683443  -4.7713360805252
H    -2.2850215726603   2.0829577471538  -4.1532353918110
H    -0.9892606342753   3.2051959925139  -4.5854917335667
H    -1.6989550492144   2.1496453780473  -5.8166145479409
C    -1.9096053739985  -0.8269385079269   5.2372774019485
H    -1.8730938416183  -1.9157634078951   5.1986764605346
H    -1.5629099527925  -0.5125717843390   6.2218512212214
H    -2.9465170241487  -0.5126306418706   5.1188520156529
C     0.7055071531436   3.1039291474455   3.3878074482962
H     0.6996321098764   3.5926125742168   2.4126899821207
H     0.0720511304863   3.6841458846608   4.0587668370105
H     1.7245462284362   3.1151446899290   3.7742478882927
"""


In the box below you can input the smiles code of the molecule. If you want to generate forcefields for molecular dynamics, enter the name that you want for your files and a three letter residue name. This code will generate a GAFF2 force field with RESP-charges calculated at the HF/sto-3g level of theory. Since this is a quantum chemical computations it may take some time if your molecule is larger.

(If you want a different basis set you can change the basis = vlx.MolecularBasis.read(molecule, 'sto-3g') line in the "Generate a force field" cell below)

In [9]:
#@title Define your molecule, either asa SMILES code or xyz-coordinates above
#@markdown - Enter the SMILES code
smiles_code = 'CC(C)CC1=CC=C(C=C1)C(C)C(=O)O' #@param {type:"string"}
#@markdown - If you want to use the xyz coordinates in the previous cell check box
use_xyz = False # @param {type:"boolean"}
if use_xyz == False:
  molecule = vlx.Molecule.read_smiles(smiles_code)
else:
  molecule = vlx.Molecule.read_str(xyz_coordinates)
print('Structure of the molecule entered: ')
molecule.show(atom_indices=True)

Structure of the molecule entered: 


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [10]:
%%capture
#@title Generate a force field
#@markdown - Run this cell to generate the force field
#@markdown - Example timing: Ibuprofen (1 min)
#@markdown - The files can be found on the left panel ("Files")
basis = vlx.MolecularBasis.read(molecule, 'def2-sv(p)')
scf_drv = vlx.ScfRestrictedDriver()
scf_drv.xcfun = "blyp"
scf_drv.ri_coulomb = True
scf_drv.grid_level = 2
scf_drv.level_shifting = 1.0
scf_drv.conv_thresh = 1e-3
scf_drv.dispersion = True
results = scf_drv.compute(molecule, basis)
ff_gen = vlx.MMForceFieldGenerator()
ff_gen.create_topology(molecule, basis, results)


In [ ]:
#@title Run dynamics
#@markdown - Example timings: 200 ps MD of ibuprofen (22 sec)
opm_dyn = vlx.OpenMMDynamics()
opm_dyn.create_system_from_molecule(molecule, ff_gen)
conformers_dict = opm_dyn.conformational_sampling(temperature=1000, ensemble='NVT', nsteps=100000, snapshots=20)

* Info * System parameters written to residue_system.xml                                                                  
* Info * System coordinates written to residue_system.pdb                                                                 
* Info * Minimized energy: -50.00041484832764                                                                             
* Info * Saved coordinates for step 5000                                                                                  
* Info * Minimized energy: -51.17293334007263                                                                             
* Info * Saved coordinates for step 10000                                                                                 
* Info * Minimized energy: -57.301178336143494                                                                            
* Info * Saved coordinates for step 15000                                                                                 


In [8]:
#@title Visualize lowest conformers
for mol, e in zip(conformers_dict['molecules'], conformers_dict['energies']):
    mol.show()
    print(e,'kJ/mol')

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

-38.910484313964844 kJ/mol


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

-33.696510314941406 kJ/mol


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

-31.41962432861328 kJ/mol


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

-27.533355712890625 kJ/mol


In [ ]:
#@title MM optimization of all snapshots
#@markdown 10 snapshots of ibuprofen (3 min)
import MDAnalysis as mda

for i in range(snapshots):
  universe = mda.Universe(gromacs_file + '.pdb')
  universe.trajectory[i]
  allatoms = universe.select_atoms('all')
  allatoms.write(gromacs_file + str(i+1) + '.xyz')
  #with mda.Writer(gromacs_file + '.xyz') as xyz:
  #   xyz.write(universe)

total_results = []
for i in range(snapshots):
  molecule = vlx.Molecule.read_xyz_file(gromacs_file + str(i+1) + '.xyz')
  print('Optimizing structure ' + str(i+1) + ' of ' + str(snapshots))
  vlxopenmm = vlx.OpenMMDriver()
  vlxopenmm.add_topology(gromacs_file + '.top')
  opt = vlx.OptimizationDriver(vlxopenmm)
  opt.ostream.mute()
#    opt.constraints = ["freeze dihedral 18 14 12 11"]
  results = opt.compute(molecule)

  total_results.append(results)

print(' ')

last_energies = []
final_geometries = []

for i in range(snapshots):
  res = total_results[i]['opt_energies'][-1]
  last_energies.append(res)
for i in range(snapshots):
  geos = total_results[i]['final_geometry']
  final_geometries.append(geos)
for i in range(snapshots):
  print('Energy', last_energies[i])

m = min(last_energies)
m_index = last_energies.index(m)
print(final_geometries[m_index])
lowest_geometry = vlx.Molecule.read_xyz_string(final_geometries[m_index])
lowest_geometry.show()

In [ ]:
#@title XTB optimization of all snapshots
#@markdown Example timing: ibuprofen (7 min)
import MDAnalysis as mda


for i in range(snapshots):
  universe = mda.Universe(gromacs_file + '.pdb')
  universe.trajectory[i]
  allatoms = universe.select_atoms('all')
  allatoms.write(gromacs_file + str(i+1) + '.xyz')
  #with mda.Writer(gromacs_file + '.xyz') as xyz:
  #   xyz.write(universe)

total_results = []
for i in range(snapshots):
  print('Optimizing structure ' + str(i+1) + ' of ' + str(snapshots))
  molecule = vlx.Molecule.read_xyz_file(gromacs_file + str(i+1) + '.xyz')
  vlxXtb = vlx.XtbDriver()
  opt = vlx.OptimizationDriver(vlxXtb)
  opt.ostream.mute()
#    opt.constraints = ["freeze dihedral 18 14 12 11"]
  results = opt.compute(molecule)

  total_results.append(results)

print(' ')
last_energies = []
final_geometries = []

for i in range(snapshots):
  res = total_results[i]['opt_energies'][-1]
  last_energies.append(res)
for i in range(snapshots):
  geos = total_results[i]['final_geometry']
  final_geometries.append(geos)
for i in range(snapshots):
  print('Energy', last_energies[i])

m = min(last_energies)
m_index = last_energies.index(m)
print(final_geometries[m_index])
lowest_geometry = vlx.Molecule.read_xyz_string(final_geometries[m_index])
lowest_geometry.show()

In [ ]:
#@title Print relative energies of all structures
for i in range(snapshots):
  lowest_geometry = vlx.Molecule.read_xyz_string(final_geometries[i])
  print('Relative energy: ', (last_energies[i] - last_energies[m_index])*627.5)
  lowest_geometry.show()
  print(final_geometries[i])


In [ ]:
#@title Write optimized coordinates to files
for i in range(snapshots):
  optimized_geometry = vlx.Molecule.read_xyz_string(final_geometries[i])
  optimized_geometry.write_xyz_file('opt' + str(i+1) + '.xyz')